# 9. Launching the full training run on Azure ML  

This cell submits a **command job** that trains the YOLO 11 segmentation model on the dataset we registered earlier.

**Key pieces**

| Element | Purpose | Value in this demo |
|---------|---------|--------------------|
| **Environment** | Re-uses the Docker image we built in step 6. | `${environment_name}@latest` |
| **Compute** | GPU (or CPU) cluster created in step 7. | `${training_gpu_cluster}` |
| **Inputs** | 1) Data asset (`uri_folder`)  2) Pre-trained weight file (`uri_file`). | From `config.yaml` |
| **Command string** | Copies & patches `data.yaml`, then calls `train.py`. | See below |
| **Outputs** | A folder that will capture checkpoints, logs, etc., and is uploaded back to the workspace datastore. |

> **How the command works**  
> 1. `cp`: copy the dataset’s `data.yaml` into a tmp dir inside the container.  
> 2. `sed`: replace the `path:` line so Ultralytics knows the *remote* location of the images.  
> 3. `python train.py`: your custom training script in `../src/`.

Feel free to bump `epochs`, `batch`, or pass extra flags such as `--imgsz 640` once the pipeline is working.


In [1]:
import yaml

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml import Input, Output
from azure.ai.ml import command

## 1) Load settings

In [2]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [3]:
subscription_id = config["azure"]["subscription_id"]
resource_group_name = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]

environment_name = config["azure"]["environment_name"]
training_gpu_cluster = config["azure"]["training_gpu_cluster"]

data_asset_path = config["train"]["data_asset_path"]

## 2) Connect to workspace

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group_name,
    workspace_name
)

## 3) Define inputs / outputs

In [5]:
data_input = Input(
    path=config["train"]["data_asset_path"],
    type="uri_folder"
)

model_input = Input(
    path="../train/pretrained_models/yolo11n-seg.pt",
    type="uri_file"
)

In [6]:
output = Output(
    type="uri_folder",
    mode="upload"
)

## 4) Command to run inside the container

In [13]:
command_job = """cp ${{inputs.data}}/data.yaml /tmp/data.yaml && \
sed -i "s|path:.*$|path: ${{inputs.data}}|" /tmp/data.yaml && \
python train.py \
--model ${{inputs.model}} \
--epochs 1 \
--batch 2 \
--output_dir ${{outputs.output_dir}}"""

In [14]:
job = command(
    code="../src",  # carpeta donde está train.py
    command=command_job,
    inputs={
        "data": data_input,
        "model": model_input
    },
    outputs={
        "output_dir": output
    },
    environment=f"{environment_name}@latest",
    compute=training_gpu_cluster,
    experiment_name="yolo11-segmentation-training",
    display_name="yolo11-segmentation-training"
)

## 5) Submit the job

In [ ]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)

In [22]:
config["train"]["job_name"] = job['name']

In [23]:
with open("../config.yaml", "w") as f:
    yaml.safe_dump(config, f, default_flow_style=False)